# Цель работы: 
Научить улучшать качество модели и научиться применять алгоритмы улучшения

## Задача:
Создать модель классификатор, которая по определенному обучаемому набору (salary, city, age, vacation_prefer, transport_prefer) будет определять предпочтения человека (target), в каком городе провести отпуск.


 ## 1)Создание синтетического датасета и подготовка данных

In [42]:
import pandas as pd
import random

# Создание синтетического датасета
data = {
    'salary': [random.randint(30000, 80000) for _ in range(1000)],
    'city': [random.choice(['Bishkek', 'London', 'Moscow', 'New York']) for _ in range(1000)],
    'age': [random.randint(30, 65) for _ in range(1000)],
    'vacation_prefer': [random.choice(['Shopping', 'Beach holiday', 'Mountain adventure']) for _ in range(1000)],
    'transport_prefer': [random.choice(['auto', 'plane']) for _ in range(1000)],
    'target': [random.choice(['London', 'Moscow', 'New York']) for _ in range(1000)]
}

df = pd.DataFrame(data)

df_encoded = pd.get_dummies(df, columns=['city', 'vacation_prefer', 'transport_prefer'])


## 2)Разделение на обучающую и тестовую выборки

In [43]:
X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3) Предсказание на случайных данных без улучшения выбирая модель классификатора Random Forest Classifier

In [44]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)

clf.fit(X_train, y_train)

# Оценка модели
accuracy = clf.score(X_test, y_test)
print(f"Точность модели на тестовой выборке: {accuracy}")

random_data = {
    'salary': [random.randint(30000, 80000) for _ in range(len(X_train))],
    'city': [random.choice(['Bishkek', 'London', 'Moscow', 'New York']) for _ in range(len(X_train))],
    'age': [random.randint(30, 65) for _ in range(len(X_train))],
    'vacation_prefer': [random.choice(['Shopping', 'Beach holiday', 'Mountain adventure']) for _ in range(len(X_train))],
    'transport_prefer': [random.choice(['auto', 'plane']) for _ in range(len(X_train))]
}

random_df = pd.DataFrame(random_data)

random_encoded = pd.get_dummies(random_df, columns=['city', 'vacation_prefer', 'transport_prefer'])

# Предсказание на случайных данных
prediction = clf.predict(random_encoded)
print(f"Предсказанный город для отпуска: {prediction}")

Точность модели на тестовой выборке: 0.28
Предсказанный город для отпуска: ['New York' 'Moscow' 'London' 'Moscow' 'London' 'Moscow' 'Moscow' 'Moscow'
 'New York' 'London' 'Moscow' 'Moscow' 'New York' 'New York' 'London'
 'London' 'London' 'New York' 'London' 'New York' 'London' 'London'
 'London' 'Moscow' 'London' 'London' 'Moscow' 'London' 'New York' 'London'
 'London' 'London' 'New York' 'London' 'London' 'London' 'New York'
 'London' 'London' 'New York' 'New York' 'London' 'London' 'Moscow'
 'Moscow' 'London' 'London' 'New York' 'London' 'New York' 'London'
 'Moscow' 'Moscow' 'London' 'Moscow' 'New York' 'New York' 'London'
 'London' 'New York' 'Moscow' 'New York' 'Moscow' 'Moscow' 'London'
 'New York' 'London' 'Moscow' 'New York' 'Moscow' 'London' 'London'
 'London' 'London' 'London' 'New York' 'New York' 'London' 'Moscow'
 'London' 'New York' 'Moscow' 'London' 'Moscow' 'Moscow' 'London' 'Moscow'
 'London' 'New York' 'London' 'Moscow' 'Moscow' 'New York' 'London'
 'London' 'Moscow'

## 4)Улучшение модели с помощью решетчатого поиска

In [45]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10]
}

clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(clf, param_grid, cv=5)

grid_search.fit(X_train, y_train)

# Получаем лучшие параметры и лучшую оценку
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Лучшие параметры:", best_params)
print("Лучшая оценка на кросс-валидации:", best_score)


Лучшие параметры: {'max_depth': None, 'min_samples_split': 10, 'n_estimators': 100}
Лучшая оценка на кросс-валидации: 0.34625000000000006


## 5)Оценка улучшенной модели на тестовой выборке и предсказание на случайных данных

In [46]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

accuracy = best_model.score(X_test, y_test)
print(f"Точность модели на тестовой выборке после улучшения: {accuracy}")

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=42)

clf.fit(X_train, y_train)

random_data = {
    'salary': [random.randint(30000, 80000) for _ in range(len(X_train))],
    'city': [random.choice(['Bishkek', 'London', 'Moscow', 'New York']) for _ in range(len(X_train))],
    'age': [random.randint(30, 65) for _ in range(len(X_train))],
    'vacation_prefer': [random.choice(['Shopping', 'Beach holiday', 'Mountain adventure']) for _ in range(len(X_train))],
    'transport_prefer': [random.choice(['auto', 'plane']) for _ in range(len(X_train))]
}

random_df = pd.DataFrame(random_data)

random_encoded = pd.get_dummies(random_df, columns=['city', 'vacation_prefer', 'transport_prefer'])

# Предсказание на случайных данных
prediction = best_model.predict(random_encoded)
print(f"Предсказанный город для отпуска: {prediction}")

Точность модели на тестовой выборке после улучшения: 0.325
Точность модели на тестовой выборке: 0.28
Предсказанный город для отпуска: ['Moscow' 'Moscow' 'New York' 'Moscow' 'Moscow' 'London' 'Moscow' 'London'
 'Moscow' 'London' 'Moscow' 'London' 'London' 'New York' 'Moscow' 'Moscow'
 'Moscow' 'Moscow' 'New York' 'London' 'Moscow' 'New York' 'Moscow'
 'New York' 'London' 'London' 'New York' 'London' 'Moscow' 'Moscow'
 'Moscow' 'Moscow' 'Moscow' 'London' 'New York' 'Moscow' 'New York'
 'New York' 'London' 'London' 'London' 'Moscow' 'London' 'Moscow'
 'New York' 'Moscow' 'New York' 'New York' 'London' 'Moscow' 'Moscow'
 'Moscow' 'London' 'Moscow' 'London' 'London' 'London' 'Moscow' 'London'
 'Moscow' 'Moscow' 'New York' 'New York' 'London' 'London' 'New York'
 'London' 'Moscow' 'London' 'Moscow' 'London' 'Moscow' 'Moscow' 'Moscow'
 'London' 'Moscow' 'London' 'London' 'New York' 'London' 'London' 'Moscow'
 'New York' 'Moscow' 'London' 'London' 'London' 'London' 'Moscow' 'London'
 'Moscow' 

# Вывод:

Мы создали модель классификатор, которая по определенному обучаемому набору (salary, city, age, vacation_prefer, transport_prefer) будет определять предпочтения человека (target), в каком городе провести отпуск.

Модель без улучшений показала результат точности 28%
Модель с улучшениями показала результат точности 33.5%
Мы улучшали модель, используя несколько методов:

    - Перекрестная проверка
    - Решетчатый поиск
    - Оценка модели
    - Использование метрик

Небольшая разница, но можем подвести итог что улучшения в правильном применении помогают в построении лучшей модели
